In [127]:
import os
import urllib.request


In [128]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [129]:
%pwd

'c:\\Users\\sr\\OneDrive\\Documents\\Work\\Agni\\Wine Quality ML Project\\MLProject_Winequality'

In [130]:
%pwd

'c:\\Users\\sr\\OneDrive\\Documents\\Work\\Agni\\Wine Quality ML Project\\MLProject_Winequality'

In [131]:
print("Current Working Directory:", os.getcwd())
print("Looking for file at:", config_filepath)

Current Working Directory: c:\Users\sr\OneDrive\Documents\Work\Agni\Wine Quality ML Project\MLProject_Winequality
Looking for file at: c:\Users\sr\OneDrive\Documents\Work\Agni\Wine Quality ML Project\MLProject_Winequality\config.yaml


In [132]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [133]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    print("123456:", config_filepath)
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

123456: c:\Users\sr\OneDrive\Documents\Work\Agni\Wine Quality ML Project\MLProject_Winequality\config.yaml


In [134]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [135]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        try:
            print(f"Downloadfile 1: {self.config.local_data_file}")
            print(f"Downloadfile 2: {self.config.source_URL}")
        
            # Check if the file already exists
            if not os.path.exists(self.config.local_data_file):
                # Attempt to download the file
                print("Starting the download...")
                filename, headers = urllib.request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                print("Download completed.")
                logger.info(f"{filename} downloaded successfully! Info: \n{headers}")
            else:
                # Log that the file already exists and display its size
                file_size = get_size(Path(self.config.local_data_file))
                print(f"File already exists. Size: {file_size}")
                logger.info(f"File already exists with size: {file_size}")

        except Exception as e:
            # Handle exceptions (e.g., HTTP errors, file errors)
            print(f"An error occurred during file download: {str(e)}")
            logger.error(f"Error during download: {str(e)}")
            raise



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

  

In [136]:
try:

    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

Reached1
[2024-09-15 14:26:02,752: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-15 14:26:02,755: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-15 14:26:02,758: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-15 14:26:02,761: INFO: common: created directory at: artifacts]
Reached2
[2024-09-15 14:26:02,763: INFO: common: created directory at: artifacts/data_ingestion]
Reached3
Reached4
Downloadfile 1: artifacts/data_ingestion/data.zip
Downloadfile 2: https://github.com/sure-AI/MLProject_Winequality/raw/main/winequality-data.zip
Starting the download...
Download completed.
[2024-09-15 14:26:03,537: INFO: 3188608278: artifacts/data_ingestion/data.zip downloaded successfully! Info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab